In [ ]:
import sys
sys.path.insert(0, '../Utilities/')

from collections import OrderedDict
from utils import *

np.random.seed(1234)
torch.manual_seed(1234)

In [ ]:
# CUDA support
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

#step functions
#step_func_1 if greater than 0 then 1, otherwise 0

def step_func_1 (a):
    a[a>=0] = 1
    a[a< 0] = 0
    return a

#step_func if greater than .5 then 1, otherwise 0

def step_func(a):
    a[a>=.5] = 1
    a[a<.5] = 0
    return a

def neg_to_zero(a):
    a[a< 0] = 0.0
    return a


#sigmoid activation function
def sigmoid_cont( a, slope):

    return 1. / (1 + torch.exp(-slope*a))


In [ ]:
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()

        # parameters
        self.depth = len(layers) - 1

        # set up layer order dict
        self.activation = torch.nn.Tanh

        layer_list = list()
        for i in range(self.depth - 1):
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1],bias=True))
            )
            layer_list.append(('activation_%d' % i, self.activation()))

        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )

        # deploy layers
        self.layers = torch.nn.Sequential(OrderedDict(layer_list))

    def forward(self, x):
        out = self.layers(x)
        return out

In [ ]:
class PhysicsInformedNN():
    def __init__(self, u, layers, Adj, thr, size, slope, cost_weight, u_cost_weight):

        self.Adj = Adj
        self.thr = thr
        self.size = size
        self.slope = slope
        self.cost_weight = cost_weight
        self.u_cost_weight = u_cost_weight

        self.t_u = torch.tensor([0]).float().to(device)
        self.t_u.requires_grad_()
        self.t_f = torch.tensor([i for i in range(1,1000)]).float().to(device)   #time step
        self.t_f = self.t_f.reshape(1,-1).t()
        self.t_f.requires_grad_()
        self.u_cost_weight = u_cost_weight
        self.u = u.clone().detach()
        self.u.requires_grad_()

        self.layers = layers

        # deep neural networks
        self.dnn = DNN(layers).to(device)
        self.best_loss = 10
        self.best_epoch = -1

        self.optimizer = torch.optim.Adam(self.dnn.parameters(), lr=0.0001)
        self.iter = 0

    def net_u(self, t):
        u = self.dnn(t)

        return u #return the neural network

    def net_f(self, t):
        """ The pytorch autograd version of calculating residual """

        u = sigmoid_cont(self.net_u(t),self.slope)

        #creates gradient used for loss function
        for i in range(self.size):

            du_dt = torch.autograd.grad(
                u[:,i], t,
                grad_outputs=torch.ones_like(u[:,i]),
                retain_graph=True,
                create_graph=True
            )[0]

            if i == 0:
                du_dt_concat = du_dt
            else:
                du_dt_concat = torch.cat((du_dt_concat, du_dt), dim=1)

        return du_dt_concat

    def loss_func(self):
        self.optimizer.zero_grad()
        u_pred = sigmoid_cont(self.net_u(self.t_u), self.slope)

        #loss for timestep 0, boundary condition
        loss_u = torch.mean((self.u - u_pred) ** 2)


        #loss for differential equation
        du_dt = self.net_f(self.t_f)
        u_t_1 = sigmoid_cont(self.net_u(self.t_f-1),self.slope)

        #differential equation
        f_temp =  neg_to_zero(torch.sub(step_func_1(torch.sub(torch.mm(self.Adj,u_t_1.t()),self.thr)),u_t_1.t()))

        loss_f = torch.mean((du_dt - f_temp.t()) ** 2)

        loss = self.u_cost_weight *loss_u + self.cost_weight*loss_f # total loss

        loss.backward()
        self.iter += 1

        if self.iter % 50 == 0:
            print(
                'Iter %d, Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), loss_u.item(), loss_f.item())
            )

        #save the best lost
        if loss < self.best_loss:
            self.best_loss = loss
            self.best_epoch = self.iter
            #saves parameters for best loss
            torch.save(self.dnn.state_dict(), f'weights.pt')

        return loss

    def train(self):
        for i in range(1000):
            self.dnn.train()

            # Backward and optimize
            self.optimizer.step(self.loss_func)

        #loads best weight
        self.dnn.load_state_dict(torch.load(f'weights.pt'))
        self.dnn.eval()


    def predict(self, T):
        T.requires_grad= True

        self.dnn.eval()
        u = sigmoid_cont(self.net_u(T),self.slope)
        du_dt = self.net_f(T)
        u = u.detach().cpu().numpy()
        du_dt = du_dt.detach().cpu().numpy()
        return u, du_dt

In [ ]:
size= 10 #size of graph
layers = [1, 128, 128, 128, 128, 128, 128, 128, 128, size] #amount of layers and neurons

data, adj = load_data_nolabel(path='data_10.csv',size=size,weighted=False) #gets the adjacency matrix
adj = adj.to_dense().type(dtype = torch.float32)
#adj = torch.nan_to_num(adj,nan=0.0, posinf=0.0, neginf=0.0)

u_train = [0 for i in range(size)]
u_train[0] = 1  #activates node 0
u_train[9] = 1  #activates node 9
u_train = torch.Tensor(u_train).type(dtype = torch.float32)
u_train = u_train.clone().detach().reshape(1,-1).t()

rng = np.random.RandomState(80)

# gets threshold
thresholds =  torch.from_numpy(rng.rand(size)).type(dtype = torch.float32)
thresholds = thresholds.reshape(1,-1).t()
thresholds = (1-u_train)*thresholds #zeros the threshold for activated nodes
print('Thresholds: ', thresholds)

Loading german dataset...
    src  dst
0   0.0  1.0
1   0.0  2.0
2   0.0  3.0
3   1.0  0.0
4   1.0  2.0
5   1.0  4.0
6   2.0  1.0
7   2.0  0.0
8   2.0  5.0
9   2.0  3.0
10  3.0  2.0
11  3.0  0.0
12  3.0  6.0
13  4.0  1.0
14  4.0  5.0
15  4.0  7.0
16  5.0  4.0
17  5.0  2.0
18  5.0  8.0
19  5.0  6.0
20  6.0  3.0
21  6.0  5.0
22  6.0  9.0
23  7.0  4.0
24  7.0  8.0
25  8.0  7.0
26  8.0  5.0
27  8.0  9.0
28  9.0  8.0
29  9.0  6.0
Thresholds:  tensor([[0.0000],
        [0.6994],
        [0.2699],
        [0.6745],
        [0.9081],
        [0.6852],
        [0.2117],
        [0.3173],
        [0.2673],
        [0.0000]])


In [ ]:
total_u_pred =np.zeros((2,size),dtype=float)
total_nan = 0
slope = 50
cost_weight = 10
u_cost_weight = 1
t_start = torch.tensor([0,10]).float().to(device).reshape(1,-1).t()
print('t:', t_start)
for i in range(1):
    model = PhysicsInformedNN(u_train.t(), layers, adj, thresholds, size, slope, cost_weight, u_cost_weight)
    model.train()
    u_pred, f_pred = model.predict(t_start)
    print(u_pred)
    check_nan = np.isnan(u_pred)

    if True not in check_nan:
        total_u_pred+= u_pred
    else:
        total_nan+=1
    print(total_u_pred)

t: tensor([[ 0.],
        [10.]])
Iter 50, Loss: 9.46517e-02, Loss_u: 8.91540e-02, Loss_f: 5.49769e-04
Iter 100, Loss: 5.72800e-03, Loss_u: 6.84020e-04, Loss_f: 5.04398e-04
Iter 150, Loss: 3.70158e-03, Loss_u: 8.69358e-04, Loss_f: 2.83223e-04
Iter 200, Loss: 2.44633e-03, Loss_u: 6.43887e-04, Loss_f: 1.80244e-04
Iter 250, Loss: 1.69984e-03, Loss_u: 4.28673e-04, Loss_f: 1.27116e-04
Iter 300, Loss: 1.17741e-03, Loss_u: 2.94085e-04, Loss_f: 8.83324e-05
Iter 350, Loss: 8.04607e-04, Loss_u: 2.19091e-04, Loss_f: 5.85517e-05
Iter 400, Loss: 5.83646e-04, Loss_u: 1.74270e-04, Loss_f: 4.09376e-05
Iter 450, Loss: 4.48059e-04, Loss_u: 1.39847e-04, Loss_f: 3.08211e-05
Iter 500, Loss: 3.57954e-04, Loss_u: 1.13447e-04, Loss_f: 2.44507e-05
Iter 550, Loss: 2.94693e-04, Loss_u: 9.33858e-05, Loss_f: 2.01307e-05
Iter 600, Loss: 2.48413e-04, Loss_u: 7.80179e-05, Loss_f: 1.70395e-05
Iter 650, Loss: 2.13428e-04, Loss_u: 6.60646e-05, Loss_f: 1.47364e-05
Iter 700, Loss: 1.86264e-04, Loss_u: 5.66135e-05, Loss_f:

In [ ]:
total_u_pred = total_u_pred/(1-total_nan)
print(total_u_pred)
print(step_func(total_u_pred[0]))
print(step_func(total_u_pred[1]))

#print(sol)

[[9.90300059e-01 3.48242139e-03 6.14477182e-03 4.25980426e-03
  5.08924900e-03 3.19760549e-03 1.45290291e-03 6.91038324e-03
  4.43801355e-06 9.95518684e-01]
 [9.97708440e-01 9.99999404e-01 9.99995112e-01 9.99998808e-01
  9.99998212e-01 9.99999762e-01 9.99938607e-01 9.99990344e-01
  9.99999642e-01 9.98857260e-01]]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
print(total_u_pred[1][:])
print([1, 0, 1, 1, 0, 1, 1, 1, 1, 1])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1, 0, 1, 1, 0, 1, 1, 1, 1, 1]
